In [4]:
import os
import pandas as pd
import time
import openpyxl
# os.chdir("./deis_emergency_care/")

In [5]:
if not os.path.isdir("./data_processing"):
    os.mkdir("./data_processing")

In [6]:
names = os.listdir("./data_temp")

In [7]:
# Extracting pieces of excel sheets to a whole excel file in a different folder
a = 0
for name in names:
    a += 1
    print(str(a) + ") " + name)
    wb = openpyxl.load_workbook("./data_temp/" + name)
    wk_region, wk_year = name.replace(".xlsx","").split("_")
    sh = wb.active

    single_letters = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
    double_letters = ["A{}".format(l) for l in single_letters] + ["BA","BB"]
    letters = single_letters + double_letters
    zero_range = ["{}{}".format(l, str(n)) for n in range(1,27) for l in letters]
    
    # Processing Function
    def process(cell_range, tag):
        new = sh[cell_range]
        new_wb = openpyxl.Workbook()
        new_sh = new_wb.active
        data = ["{}".format(cell.value) for row in new for cell in row]    
        points = zip(zero_range,data)
        for p in points:
            new_sh[p[0]] = p[1]
        
        new_wb.save("./data_processing/" + wk_region + "_" + wk_year + "_" + tag + ".xlsx")

    # Actual processing
    process("A47:BB72", "less1")
    process("A76:BB101", "1to4")
    process("A105:BB130", "5to14")
    process("A134:BB159", "15to65")
    process("A163:BB188", "65plus")


1) Maule_2010.xlsx
2) Los Lagos_2008.xlsx
3) Libertador Bernardo OHiggins_2010.xlsx
4) Araucania_2016.xlsx
5) Bio-Bio_2016.xlsx
6) Atacama_2011.xlsx
7) Los Rios_2012.xlsx
8) Aisen_2012.xlsx
9) Valparaiso_2015.xlsx
10) Metropolitana de Santiago_2017.xlsx
11) Antofagasta_2012.xlsx
12) Coquimbo_2011.xlsx
13) Tarapaca_2016.xlsx
14) Los Rios_2008.xlsx
15) Los Lagos_2012.xlsx
16) Antofagasta_2008.xlsx
17) Arica y Parinacota_2017.xlsx
18) Valparaiso_2019.xlsx
19) Magallanes_2016.xlsx
20) Aisen_2008.xlsx
21) Magallanes_2017.xlsx
22) Aisen_2009.xlsx
23) Valparaiso_2018.xlsx
24) Arica y Parinacota_2016.xlsx
25) Antofagasta_2009.xlsx
26) Los Lagos_2013.xlsx
27) Los Rios_2009.xlsx
28) Tarapaca_2017.xlsx
29) Antofagasta_2013.xlsx
30) Coquimbo_2010.xlsx
31) Metropolitana de Santiago_2016.xlsx
32) Valparaiso_2014.xlsx
33) Aisen_2013.xlsx
34) Atacama_2010.xlsx
35) Los Rios_2013.xlsx
36) Bio-Bio_2017.xlsx
37) Araucania_2017.xlsx
38) Maule_2011.xlsx
39) Los Lagos_2009.xlsx
40) Libertador Bernardo OHiggi

In [8]:
# Loading filenames
files = os.listdir("./data_processing")

In [9]:
# Emergency dimension
e_dim = pd.read_csv("./data_final/emergency_table.csv")

# Age range dimension
age_dim = pd.read_csv("./data_final/age_table.csv")

# Geography dimension
geo_dim = pd.read_csv("./data_final/geography_table.csv")

In [10]:
# Date dimension
date_dim = pd.read_csv("./data_final/date_table.csv", dtype = "object")

In [11]:
# Tidying dataframes

df_list = []

a = 0
for file in files:
    a += 1
    print("{}) {}".format(a, file))
    df = pd.read_excel("./data_processing/" + file)
    f_region, f_year, f_age = file.replace(".xlsx","").split("_")
    df = df.drop(labels = [0,1,8,14,19])

    df = df.drop(columns = ["Total"]).rename(columns = {"None":"emergency"})
    df = pd.melt(df, id_vars = "emergency", var_name = "week", value_name = "total")
    df = df.merge(e_dim[["emergency_id", "emergency_index"]], left_on = "emergency", right_on = "emergency_index", how = "left")
    df = df.drop(columns = ["emergency", "emergency_index"])

    df["year"] = f_year
    df["date"] = df.year + "-" + df.week
    df = df.drop(columns = ["week", "year"])
    df = df.merge(date_dim[["date_id", "date_index"]], left_on = "date", right_on = "date_index", how = "left")
    df = df.drop(columns = ["date","date_index"])

    df["age"] = f_age
    df = df.merge(age_dim[["age_id", "age_index"]], left_on = "age", right_on = "age_index", how = "left")
    df = df.drop(columns = ["age", "age_index"])

    df["region"] = f_region
    df = df.merge(geo_dim[["geography_id", "geography_index"]], left_on = "region", right_on = "geography_index", how = "left")
    df = df.drop(columns = ["region", "geography_index"])

    if df["date_id"].isnull().any() == True:
        print("NULL DATE VALUES")

    df_list.append(df)


1) Antofagasta_2013_65plus.xlsx
2) Antofagasta_2008_15to65.xlsx
NULL DATE VALUES
3) Aisen_2019_less1.xlsx
NULL DATE VALUES
4) Los Lagos_2015_65plus.xlsx
5) Los Rios_2016_15to65.xlsx
6) Bio-Bio_2009_1to4.xlsx
7) Maule_2010_15to65.xlsx
8) Valparaiso_2018_1to4.xlsx
9) Los Rios_2013_1to4.xlsx
10) Coquimbo_2015_5to14.xlsx
11) Valparaiso_2018_5to14.xlsx
12) Tarapaca_2015_15to65.xlsx
13) Arica y Parinacota_2013_5to14.xlsx
14) Los Lagos_2012_1to4.xlsx
15) Antofagasta_2013_1to4.xlsx
16) Araucania_2011_less1.xlsx
17) Magallanes_2019_5to14.xlsx
NULL DATE VALUES
18) Araucania_2016_15to65.xlsx
19) Magallanes_2014_15to65.xlsx
20) Coquimbo_2018_1to4.xlsx
21) Atacama_2009_1to4.xlsx
22) Valparaiso_2015_15to65.xlsx
23) Coquimbo_2014_15to65.xlsx
24) Maule_2008_5to14.xlsx
NULL DATE VALUES
25) Bio-Bio_2011_less1.xlsx
26) Bio-Bio_2018_65plus.xlsx
27) Los Lagos_2010_5to14.xlsx
28) Tarapaca_2013_5to14.xlsx
29) Metropolitana de Santiago_2011_1to4.xlsx
30) Magallanes_2016_less1.xlsx
31) Valparaiso_2012_65plus.x

232) Valparaiso_2014_65plus.xlsx
233) Atacama_2008_1to4.xlsx
NULL DATE VALUES
234) Coquimbo_2019_1to4.xlsx
NULL DATE VALUES
235) Coquimbo_2015_65plus.xlsx
236) Valparaiso_2015_less1.xlsx
237) Bio-Bio_2019_15to65.xlsx
NULL DATE VALUES
238) Atacama_2013_less1.xlsx
239) Magallanes_2015_65plus.xlsx
240) Araucania_2017_65plus.xlsx
241) Antofagasta_2012_15to65.xlsx
242) Los Lagos_2014_15to65.xlsx
243) Antofagasta_2009_65plus.xlsx
244) Antofagasta_2010_5to14.xlsx
245) Coquimbo_2018_less1.xlsx
246) Bio-Bio_2008_1to4.xlsx
NULL DATE VALUES
247) Aisen_2014_5to14.xlsx
248) Maule_2016_5to14.xlsx
249) Los Rios_2017_65plus.xlsx
250) Valparaiso_2019_1to4.xlsx
NULL DATE VALUES
251) Maule_2011_65plus.xlsx
252) Los Rios_2012_1to4.xlsx
253) Metropolitana de Santiago_2013_less1.xlsx
254) Maule_2016_15to65.xlsx
255) Maule_2016_1to4.xlsx
256) Maule_2019_less1.xlsx
NULL DATE VALUES
257) Los Rios_2010_15to65.xlsx
258) Atacama_2019_65plus.xlsx
NULL DATE VALUES
259) Magallanes_2008_less1.xlsx
NULL DATE VALUES
26

459) Magallanes_2010_5to14.xlsx
460) Maule_2013_65plus.xlsx
461) Maule_2008_15to65.xlsx
NULL DATE VALUES
462) Araucania_2018_less1.xlsx
463) Atacama_2017_5to14.xlsx
464) Tarapaca_2012_1to4.xlsx
465) Los Lagos_2016_15to65.xlsx
466) Antofagasta_2010_15to65.xlsx
467) Valparaiso_2011_5to14.xlsx
468) Coquimbo_2017_65plus.xlsx
469) Valparaiso_2016_65plus.xlsx
470) Antofagasta_2014_less1.xlsx
471) Arica y Parinacota_2009_less1.xlsx
472) Maule_2015_1to4.xlsx
473) Araucania_2015_65plus.xlsx
474) Magallanes_2017_65plus.xlsx
475) Metropolitana de Santiago_2017_5to14.xlsx
476) Tarapaca_2016_65plus.xlsx
477) Arica y Parinacota_2019_15to65.xlsx
NULL DATE VALUES
478) Maule_2012_less1.xlsx
479) Aisen_2010_less1.xlsx
480) Tarapaca_2009_less1.xlsx
481) Tarapaca_2011_15to65.xlsx
482) Metropolitana de Santiago_2018_less1.xlsx
483) Libertador Bernardo OHiggins_2009_less1.xlsx
484) Metropolitana de Santiago_2019_15to65.xlsx
NULL DATE VALUES
485) Magallanes_2010_15to65.xlsx
486) Los Rios_2011_1to4.xlsx
487) 

690) Maule_2012_15to65.xlsx
691) Maule_2009_65plus.xlsx
692) Metropolitana de Santiago_2012_1to4.xlsx
693) Arica y Parinacota_2017_less1.xlsx
694) Aisen_2019_65plus.xlsx
NULL DATE VALUES
695) Los Lagos_2017_65plus.xlsx
696) Libertador Bernardo OHiggins_2018_5to14.xlsx
697) Antofagasta_2011_65plus.xlsx
698) Tarapaca_2013_1to4.xlsx
699) Tarapaca_2017_less1.xlsx
700) Los Lagos_2014_less1.xlsx
701) Bio-Bio_2015_5to14.xlsx
702) Metropolitana de Santiago_2009_5to14.xlsx
703) Antofagasta_2010_1to4.xlsx
704) Libertador Bernardo OHiggins_2017_less1.xlsx
705) Los Rios_2014_less1.xlsx
706) Los Lagos_2011_1to4.xlsx
707) Tarapaca_2018_5to14.xlsx
708) Magallanes_2012_5to14.xlsx
709) Magallanes_2008_1to4.xlsx
NULL DATE VALUES
710) Los Lagos_2010_15to65.xlsx
711) Antofagasta_2016_15to65.xlsx
712) Araucania_2018_1to4.xlsx
713) Atacama_2015_5to14.xlsx
714) Valparaiso_2013_5to14.xlsx
715) Libertador Bernardo OHiggins_2019_1to4.xlsx
NULL DATE VALUES
716) Maule_2015_65plus.xlsx
717) Los Rios_2013_65plus.xl

In [12]:
df = pd.concat(df_list, ignore_index = True)
df = df[df["total"] != "-"]
df = df.dropna()

df["date_id"] = df["date_id"].astype("int")
df = df.reset_index(drop = True)
df = df[["date_id", "geography_id", "emergency_id", "age_id", "total"]]

In [13]:
if not os.path.isdir("./data_final"):
    os.mkdir("./data_final")
    
df.to_csv("./data_final/fact_emergency.csv", index = False)